# OpenAI Function Calling In LangChain 🤖

## Introduction
This notebook demonstrates how to `use the LangChain framework to integrate OpenAI function calling capabilities`. We'll cover the setup, the syntax of Pydantic data classes, and how to convert these data classes into OpenAI functions. Additionally, we'll explore how to bind and invoke these functions within LangChain models.

### Setup the Environment, OpenAI API Key  and Imports
First, we need to import the necessary libraries and set up the Environemnt and the OpenAI API key:

In [1]:
# Import necessary libraries
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

**Note** Ensure you have the required packages installed:
```py
%pip install pydantic==1.10.8
%pip install rich
```

In [2]:
# Import necessary modules from Pydantic 
from typing import List
from pydantic import BaseModel, Field

In [4]:
# Import necessary modules from rich library that helps to improve the readability of nested dictionary outputs
from rich import print
from rich.pretty import Pretty

### Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

- Creation of a class 

With Python, you can define the class as follows:

In [5]:
# In Python you can create a claas like that: 
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [6]:
# Example 1  usage
foo = User(name="Pinco",age=32, email="pinc@pall.com")
#print(foo.name)
foo.name

# Example 2 usage
foo = User(name="Pinco",age="bar", email="pinc@pall.com")
foo.age

Pinco

'Pinco'

**Note**: The above line should raise an error because 'age' should be an integer

With Pydantic, you can define the class as follows:

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
# Example usage
foo_p = pUser(name="Jane", age=32, email="jane@pall.com")
foo_p.name

'Jane'

In [10]:
class Class(BaseModel):
    students: List[pUser]

In [11]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@pall.com")]
)

In [12]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@palla.com')])

## Pydantic to OpenAI function definition


We can convert Pydantic data classes into OpenAI function definitions.

In [13]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [14]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [15]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\Michela\AppData\Local\Temp\ipykernel_14632\1313296718.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [16]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [17]:
print(Pretty(weather_function))


{
    'name': 'WeatherSearch',
    'description': 'Call this with an airport code to get the weather at that airport',
    'parameters': {
        'properties': {'airport_code': {'description': 'airport code to get weather for', 'type': 'string'}},
        'required': ['airport_code'],
        'type': 'object'
    }
}

In [18]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [19]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [20]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [21]:
response = convert_pydantic_to_openai_function(WeatherSearch2)
print(Pretty(response))

{
    'name': 'WeatherSearch2',
    'description': 'Call this with an airport code to get the weather at that airport',
    'parameters': {
        'properties': {'airport_code': {'type': 'string'}},
        'required': ['airport_code'],
        'type': 'object'
    }
}

In [22]:
from langchain.chat_models import ChatOpenAI

In [23]:
model = ChatOpenAI()

C:\Users\Michela\AppData\Local\Temp\ipykernel_14632\1603725051.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


**Note** weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [24]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 70, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-733b9aa8-d978-4b2d-a028-ec200c8e416e-0')

In [25]:
response = model.invoke("what is the weather in SF today?", functions=[weather_function])
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 70,
            'total_tokens': 88,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-fcb12282-3cd8-40b6-b491-fcb8b7c4580d-0'
)

In [26]:
model_with_function = model.bind(functions=[weather_function])

In [27]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 69, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-1d0be4fa-14e5-4848-831a-6e7751e603da-0')

In [31]:
response = model_with_function.invoke("what is the weather in sf?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 69,
            'total_tokens': 87,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-698f88f9-75aa-4fdd-bbc9-666e7cdb77cc-0'
)

## Forcing the Model to Use a Function

We can force the model to use a specific function when invoking it.

**Note** model_with_function = model.bind(functions=[weather_function])

In [28]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [29]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 79, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f795defe-e403-4580-ad12-fe21f3b9dbbf-0')

In [30]:
response = model_with_forced_function.invoke("what is the weather in Amsterdam?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 8,
            'prompt_tokens': 79,
            'total_tokens': 87,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-30030a15-cf12-4de7-a673-b6ca69c047c6-0'
)

**Explanation output**  This output is a structured representation of an AIMessage object.This output indicates that the AI model invoked a function named WeatherSearch with an argument of {"airport_code":"SFO"} and provides detailed metadata about the token usage and model.  
Interesting is to have a look at the structure of the AIMessage:   
Class: AIMessage  
Attributes:  
  - content: An empty string, indicating no direct textual content in the message.    
  - additional_kwargs: A dictionary containing additional keyword arguments:  
      - function_call: A dictionary specifying:
          - arguments: A JSON string with the argument {"airport_code":"SFO"}, indicating the airport code for San Francisco.
      - name: The function name WeatherSearch.    

response_metadata: A dictionary with metadata about the response:  
  - token_usage: A dictionary detailing token usage:
completion_tokens: 8
prompt_tokens: 79
total_tokens: 87
completion_tokens_details: A dictionary with various token counts, all set to 0.
prompt_tokens_details: A dictionary with audio_tokens and cached_tokens, both set to 0.
  - model_name: The model used is gpt-3.5-turbo.
  - system_fingerprint: None.
  - finish_reason: The reason for finishing is stop.
  - logprobs: None.  
  
id: A unique identifier for the run: run-cc94ce3d-0e5c-4f99-9a20-5cdae01e5231-0.

In [31]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 74, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-987a7b2e-dbd8-43e5-8316-6a4969b07a93-0')

In [32]:
response = model_with_forced_function.invoke("hi!")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 8,
            'prompt_tokens': 74,
            'total_tokens': 82,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-f0741863-fc21-4ac2-85b2-7d944d5fa042-0'
)

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [33]:
from langchain.prompts import ChatPromptTemplate

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [35]:
chain = prompt | model_with_function

In [36]:
chain.invoke({"input": "what is the weather in Amsterdam?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 75, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-1b11657d-9435-4c62-8d53-15fa68bf41a7-0')

In [37]:
response = chain.invoke({"input": "what is the weather in Amsterdam?"})
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 75,
            'total_tokens': 93,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-c45b9c1b-d3f8-4104-92b5-a28ef86400b2-0'
)

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [38]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

Let's convert Pydantic data classes to OpenAI function definitions and stores them in a list named `functions`. Each function can be used for specific tasks such as weather search and artist search in the OpenAI model.

In [39]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch), # Converts WeatherSearch Pydantic class to OpenAI function
    convert_pydantic_to_openai_function(ArtistSearch),  # Converts ArtistSearch Pydantic class to OpenAI function
]

Let's binds a list of predefined functions to an OpenAI model. The 'functions' list contains converted Pydantic data classes that define specific tasks (e.g., WeatherSearch, ArtistSearch). Binding these functions to the model enables the model to invoke them during its execution for specific queries.

In [40]:
model_with_functions = model.bind(functions=functions)

Next, let's  snippet invoke the `model_with_functions` with the query "what is the weather in sf?". The model is expected to use the bound functions (e.g., WeatherSearch) to process this query. The query will trigger the model to call the appropriate function to get the weather information for San Francisco (SFO).

In [41]:
model_with_functions.invoke("what is the weather in Amsterdam?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 116, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c46b2e80-be6e-40e0-b0f6-da8033b389a5-0')

In [42]:
response = model_with_functions.invoke("what is the weather in Amsterdam?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 116,
            'total_tokens': 134,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-0d9724b3-cbe2-471f-b545-f07ff525dd6c-0'
)

In [43]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 118, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-5054f35d-bb4f-419f-88fb-07d5ac80d80a-0')

In [44]:
response = model_with_functions.invoke("what are three songs by taylor swift?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {'arguments': '{"artist_name":"taylor swift","n":3}', 'name': 'ArtistSearch'}
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 23,
            'prompt_tokens': 118,
            'total_tokens': 141,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-299eb1c0-1fca-4232-bce7-2964bf70fdd2-0'
)

In [45]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 111, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c40b051b-100a-4e71-8f96-a598048a292d-0')

In [46]:
response = model_with_functions.invoke("hi!")
print(Pretty(response))

AIMessage(
    content='Hello! How can I assist you today?',
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 11,
            'prompt_tokens': 111,
            'total_tokens': 122,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-ea366545-8951-4e5e-9626-d8ea2029662e-0'
)

## Conclusion
In this notebook, we demonstrated how to `use LangChain to integrate OpenAI function calling capabilities`. We covered the setup, Pydantic data classes, converting these classes into OpenAI functions, and invoking these functions within LangChain models. These examples showcase the flexibility and power of LangChain in building advanced language models and workflows.